### 1. Price Impact (Slippage):

Most Automated Market Makers (AMMs) like Uniswap use a formula that relates the amount of tokens in the pool to the price. For Uniswap's V2, it's:

\[ P_X \times P_Y = k \]

Where:
- \( P_X \) and \( P_Y \) are the amounts of tokens X and Y in the pool.
- \( k \) is a constant value (it's constant as long as there's no token addition/removal).

Given a trade amount \( \Delta X \) (buying \( \Delta X \) of token X by selling token Y), the price impact is:

\[ \text{Price Impact} = \frac{P_X}{P_X + \Delta X} - 1 \]

### 2. Liquidity Depth:

The liquidity depth can be seen as the amount of a trade that results in a certain price impact. If you want to limit your price impact to, say, 1%, you can solve for \( \Delta X \) in the above equation.

### 3. Gas Fees:

Gas fees are more tricky as they depend on the Ethereum network congestion. However, you can estimate them. Let:

\[ G = \text{Gas cost of a trade (in ETH)} \]
\[ P_{\text{ETH}} = \text{Price of ETH (in the token you are arbitraging, e.g., USD)} \]

Then, the cost in terms of your arbitrage token is:

\[ C = G \times P_{\text{ETH}} \]

### 4. Profit Calculation:

For each trade amount \( \Delta X \), calculate the profit as:

Profit = (Price difference between pools - Price Impact) * Delta X - C

### 5. Iterative Optimization:

Given a range [min, max] for \( \Delta X \) (minimum and maximum trade amounts you're considering):

1. Start with a small \( \Delta X \) (e.g., min).
2. Calculate the profit using the above formula.
3. Increment \( \Delta X \) by a step size.
4. Repeat steps 2-3 until \( \Delta X \) = max.
5. The \( \Delta X \) with the maximum profit is your optimal trade amount.

Note: This is a simplified model and assumes you're trading in one direction (e.g., from token X to Y). In reality, arbitrage might involve trading in both directions or even involve multiple tokens. The principles remain similar, but the calculations get more involved.

In [4]:
def price_impact(P, delta):
    """Calculate price impact given liquidity P and trade amount delta"""
    return P / (P + delta) - 1

def profit(price_difference, P_A, P_B, delta, gas_cost, eth_price):
    """Calculate profit given all the parameters including price impacts for both pools"""
    impact_A = price_impact(P_A, delta)
    impact_B = price_impact(P_B, delta)
    gas_fee = gas_cost * eth_price
    return (price_difference - impact_A - impact_B) * delta - gas_fee

def optimal_trade_amount(price_difference, P_A, P_B, gas_cost, eth_price, min_trade, max_trade, step):
    """Determine the optimal trade amount for maximum profit"""
    max_profit = -float('inf')
    optimal_amount = 0
    
    # Iterate over trade sizes
    for delta in range(min_trade, max_trade + 1, step):
        current_profit = profit(price_difference, P_A, P_B, delta, gas_cost, eth_price)
        
        if current_profit > max_profit:
            max_profit = current_profit
            optimal_amount = delta
            
    return optimal_amount, max_profit

# Example usage:
price_difference = 100  # Difference in price between two pools
P_A = 100000              # Amount of token in Pool A
P_B = 150000              # Amount of token in Pool B
gas_cost = 0.005        # Estimated gas cost in ETH
eth_price = 2000        # Price of ETH in your arbitrage token
min_trade = 1           # Minimum trade amount you're considering
max_trade = 100         # Maximum trade amount you're considering
step = 1                # Step size

opt_amount, max_prof = optimal_trade_amount(price_difference, P_A, P_B, gas_cost, eth_price, min_trade, max_trade, step)
print(f"Optimal Trade Amount: {opt_amount}")
print(f"Maximum Profit: {max_prof}")


Optimal Trade Amount: 100
Maximum Profit: 9990.166522351732


In [6]:
def get_execution_price(initial_reserve, trade_amount):
    """
    Calculate the execution price for buying `trade_amount` from a Uniswap-like pool.
    Uses the constant product formula: x * y = k.
    """
    final_reserve = initial_reserve - trade_amount
    average_price = (initial_reserve - final_reserve) / trade_amount
    return average_price

def optimal_trade_amount(reserve_A, reserve_B, price_difference):
    """
    Determine the optimal trade amount for arbitrage between two pools, A and B.
    Assume you're buying from Pool A and selling to Pool B.
    """
    # Start with a small trade amount and increase iteratively.
    delta_x = 0.01
    
    while True:
        buy_price_A = get_execution_price(reserve_A, delta_x)
        sell_price_B = get_execution_price(reserve_B, -delta_x)  # negative because we're selling
        
        if buy_price_A + price_difference >= sell_price_B:
            # The price difference is no longer profitable due to slippage.
            break
        
        delta_x += 0.01
        
    return delta_x - 0.01  # subtract the last increment since it broke the loop

# Example usage:
reserve_A = 1000  # Asset amount in Pool A
reserve_B = 1100  # Asset amount in Pool B
price_difference = 10  # Assume asset x is 0.05 cheaper in Pool A than Pool B

trade_amount = optimal_trade_amount(reserve_A, reserve_B, price_difference)
print(f"Optimal trade amount for arbitrage: {trade_amount}")


Optimal trade amount for arbitrage: 0.0


In [7]:
def get_price_after_trade(reserve, delta, pool_constant):
    """
    Given an AMM pool's reserve and a trade amount (delta), calculate the effective price after trade.
    """
    new_reserve = reserve - delta
    new_other_side = pool_constant / new_reserve
    price = (new_other_side - (pool_constant / reserve)) / delta
    return price

def find_optimal_trade_amount(pool_A_reserve, pool_B_reserve, step=0.01, max_iterations=10000):
    """
    Find the optimal trade amount for arbitrage between two AMM pools.
    """
    pool_A_constant = pool_A_reserve * (pool_A_reserve + step)  # Just a small increment to get the other side
    pool_B_constant = pool_B_reserve * (pool_B_reserve - step)  # Just a small decrement to get the other side
    
    max_profit = 0
    optimal_trade = 0
    
    for i in range(1, max_iterations):
        trade_size = i * step
        buy_price_A = get_price_after_trade(pool_A_reserve, trade_size, pool_A_constant)
        sell_price_B = get_price_after_trade(pool_B_reserve, -trade_size, pool_B_constant)
        
        profit = (sell_price_B - buy_price_A) * trade_size
        
        if profit > max_profit:
            max_profit = profit
            optimal_trade = trade_size
        else:
            # If profit starts decreasing, we can break out of the loop
            break
    
    return optimal_trade

# Example
pool_A_reserve = 1000
pool_B_reserve = 1100

optimal_amount = find_optimal_trade_amount(pool_A_reserve, pool_B_reserve)
print(f"Optimal trade amount for arbitrage: {optimal_amount}")


Optimal trade amount for arbitrage: 0


In [13]:
import sympy as sp

# Define the variables
Delta_a = sp.symbols('Delta_a')
R0, R1, R1_prime, R2, R2_prime, R3, r = sp.symbols('R0 R1 R1_prime R2 R2_prime R3 r')

# Express Delta_b, Delta_c, and Delta_a_prime in terms of Delta_a
Delta_b = R1 - (R0 * R1) / (R0 + r * Delta_a)
Delta_c = R2 - (R1_prime * R2) / (R1_prime + r * Delta_b)
Delta_a_prime = R3 - (R2_prime * R3) / (R2_prime + r * Delta_c)

# Define the profit function
profit = Delta_a_prime - Delta_a

# Differentiate profit with respect to Delta_a
profit_derivative = sp.diff(profit, Delta_a)

# Solve for Delta_a
optimal_trade_amount = sp.solve(profit_derivative, Delta_a)

print(optimal_trade_amount)
# Define the arbitrary values
values = {
    R0: 1000,
    R1: 20000,
    R1_prime: 1500,
    R2: 2500,
    R2_prime: 1300,
    R3: 17000,
    r: 0.003  # 0.3% fee
}

# Evaluate the optimal trade amount
optimal_amount_evaluated = [expr.evalf(subs=values) for expr in optimal_trade_amount]

optimal_amount_evaluated


[(-R0*R1_prime*R2_prime*r - sqrt(R0*R1*R1_prime*R2*R2_prime*R3*r**5))/(R1*R2*r**4 + R1*R2_prime*r**3 + R1_prime*R2_prime*r**2), (-R0*R1_prime*R2_prime*r + sqrt(R0*R1*R1_prime*R2*R2_prime*R3*r**5))/(R1*R2*r**4 + R1*R2_prime*r**3 + R1_prime*R2_prime*r**2)]


[-321541.033948137, -319342.399215116]

In [12]:
def get_Ea_Eb(R0, R1, R1_prime, R2, r):
    Ea = R0 * R1_prime / R1_prime
    Eb = (R1_prime + R1 * r * R0) / (R2 * r)
    return Ea, Eb

def getOptimalAmount(Ea, Eb, r):
    # Ensure that the given parameters are valid for arbitrage
    if Ea >= Eb or r >= 1:
        return 0

    # Calculate the optimal input amount
    optimal_amount = (r * Ea * Eb) / (Eb - Ea)

    return optimal_amount

# Example usage
R0 = 1000
R1 = 1100
R1_prime = 1200 
R2 = 1000 
r= 0.003  # Fee rate, assuming 0.3%
Ea,Eb = get_Ea_Eb(R0, R1, R1_prime, R2, r)
optimal_amount = getOptimalAmount(Ea, Eb, r)
print("Optimal input amount:", optimal_amount)

Optimal input amount: 4.500013500040501


In [16]:
import numpy as np
import cvxpy as cp
import itertools

# Problem data
global_indices = list(range(4))

# 0 = TOKEN-0
# 1 = TOKEN-1
# 2 = TOKEN-2
# 3 = TOKEN-3

local_indices = [
    [0, 1, 2, 3], # TOKEN-0/TOKEN-1/TOKEN-2/TOKEN-3
    [0, 1], # TOKEN-0/TOKEN-1
    [1, 2], # TOKEN-1/TOKEN-2
    [2, 3], # TOKEN-2/TOKEN-3
    [2, 3] # TOKEN-2/TOKEN-3
]

reserves = list(map(np.array, [
    [4, 4, 4, 4], # balancer with 4 assets in pool TOKEN-0, TOKEN-1, TOKEN-2, TOKEN-3 (4 TOKEN-0, 4 TOKEN-1, 4 TOKEN-2 & 4 TOKEN-3 IN POOL)
    [10, 1], # uniswapV2 TOKEN-0/TOKEN-1 (10 TOKEN-0 & 1 TOKEN-1 IN POOL)
    [1, 5], # uniswapV2 TOKEN-1/TOKEN-2 (1 TOKEN-1 & 5 TOKEN-2 IN POOL)
    [40, 50], # uniswapV2 TOKEN-2/TOKEN-3  (40 TOKEN-2 & 50 TOKEN-3 IN POOL)
    [10, 10] # constant_sum TOKEN-2/TOKEN-3 (10 TOKEN-2 & 10 TOKEN-3 IN POOL)
]))

fees = [
    .998, # balancer fees
    .997, # uniswapV2 fees
    .997, # uniswapV2 fees
    .997, # uniswapV2 fees
    .999 # constant_sum fees
]

# "Market value" of tokens (say, in a centralized exchange)
market_value = [
    1.5, # TOKEN-0
    10, # TOKEN-1
    2, # TOKEN-2
    3 # TOKEN-3
] 

# Build local-global matrices
n = len(global_indices)
m = len(local_indices)

A = []
for l in local_indices: # for each CFMM
    n_i = len(l) # n_i = number of tokens avaiable for CFMM i
    A_i = np.zeros((n, n_i)) # Create matrix of 0's
    for i, idx in enumerate(l):
        A_i[idx, i] = 1
    A.append(A_i)

# Build variables

# tender delta
deltas = [cp.Variable(len(l), nonneg=True) for l in local_indices]

# receive lambda
lambdas = [cp.Variable(len(l), nonneg=True) for l in local_indices]

psi = cp.sum([A_i @ (L - D) for A_i, D, L in zip(A, deltas, lambdas)])

# Objective is to maximize "total market value" of coins out
obj = cp.Maximize(market_value @ psi) # matrix multiplication

# Reserves after trade
new_reserves = [R + gamma_i*D - L for R, gamma_i, D, L in zip(reserves, fees, deltas, lambdas)]

# Trading function constraints
cons = [
    # Balancer pool with weights 4, 3, 2, 1
    cp.geo_mean(new_reserves[0], p=np.array([4, 3, 2, 1])) >= cp.geo_mean(reserves[0]),

    # Uniswap v2 pools
    cp.geo_mean(new_reserves[1]) >= cp.geo_mean(reserves[1]),
    cp.geo_mean(new_reserves[2]) >= cp.geo_mean(reserves[2]),
    cp.geo_mean(new_reserves[3]) >= cp.geo_mean(reserves[3]),

    # Constant sum pool
    cp.sum(new_reserves[4]) >= cp.sum(reserves[4]),
    new_reserves[4] >= 0,

    # Arbitrage constraint
    psi >= 0
]

# Set up and solve problem
prob = cp.Problem(obj, cons)
prob.solve()


# Trade Execution Ordering

current_tokens = [0, 0, 0, 0]
new_current_tokens = [0, 0, 0, 0]
tokens_required_arr = []
tokens_required_value_arr = []

pool_names = ["BALANCER 0/1/2/3", "UNIV2 0/1", "UNIV2 1/2", "UNIV2 2/3", "CONSTANT SUM 2/3"]

permutations = itertools.permutations(list(range(len(local_indices))), len(local_indices))
permutations2 = []
for permutation in permutations:
    permutations2.append(permutation)
    current_tokens = [0, 0, 0, 0]
    new_current_tokens = [0, 0, 0, 0]   
    tokens_required = [0, 0, 0, 0]
    for pool_id in permutation:
        pool = local_indices[pool_id]
        for global_token_id in pool:
            local_token_index = pool.index(global_token_id)
            new_current_tokens[global_token_id] = current_tokens[global_token_id] + (lambdas[pool_id].value[local_token_index] - deltas[pool_id].value[local_token_index])

            if new_current_tokens[global_token_id] < 0 and new_current_tokens[global_token_id] < current_tokens[global_token_id]:
                if current_tokens[global_token_id] < 0:
                    tokens_required[global_token_id] += (current_tokens[global_token_id] - new_current_tokens[global_token_id])
                    new_current_tokens[global_token_id] = 0
                else:
                    tokens_required[global_token_id] += (-new_current_tokens[global_token_id])
                    new_current_tokens[global_token_id] = 0
            current_tokens[global_token_id] = new_current_tokens[global_token_id]

    tokens_required_value = []
    for i1, i2 in zip(tokens_required, market_value):
        tokens_required_value.append(i1*i2)

    tokens_required_arr.append(tokens_required)
    tokens_required_value_arr.append(sum(tokens_required_value))

min_value = min(tokens_required_value_arr)
min_value_index = tokens_required_value_arr.index(min_value)


print("\n-------------------- ARBITRAGE TRADES + EXECUTION ORDER --------------------\n")
for pool_id in permutations2[min_value_index]:
    pool = local_indices[pool_id]
    print(f"\nTRADE POOL = {pool_names[pool_id]}")

    for global_token_id in pool:
        local_token_index = pool.index(global_token_id)
        if (lambdas[pool_id].value[local_token_index] - deltas[pool_id].value[local_token_index]) < 0:  
            print(f"\tTENDERING {-(lambdas[pool_id].value[local_token_index] - deltas[pool_id].value[local_token_index])} TOKEN {global_token_id}")
    
    for global_token_id in pool:
        local_token_index = pool.index(global_token_id)
        if (lambdas[pool_id].value[local_token_index] - deltas[pool_id].value[local_token_index]) >= 0:  
            print(f"\tRECEIVEING {(lambdas[pool_id].value[local_token_index] - deltas[pool_id].value[local_token_index])} TOKEN {global_token_id}")


print("\n-------------------- REQUIRED TOKENS TO KICK-START ARBITRAGE --------------------\n")
print(f"TOKEN-0 = {tokens_required_arr[min_value_index][0]}")
print(f"TOKEN-1 = {tokens_required_arr[min_value_index][1]}")
print(f"TOKEN-2 = {tokens_required_arr[min_value_index][2]}")
print(f"TOKEN-3 = {tokens_required_arr[min_value_index][3]}")

print(f"\nUSD VALUE REQUIRED = ${min_value}")

print("\n-------------------- TOKENS & VALUE RECEIVED FROM ARBITRAGE --------------------\n")
net_network_trade_tokens = [0, 0, 0, 0]
net_network_trade_value = [0, 0, 0, 0]

for pool_id in permutations2[min_value_index]:
    pool = local_indices[pool_id]
    for global_token_id in pool:
        local_token_index = pool.index(global_token_id)
        net_network_trade_tokens[global_token_id] += lambdas[pool_id].value[local_token_index]
        net_network_trade_tokens[global_token_id] -= deltas[pool_id].value[local_token_index]

for i in range(0, len(net_network_trade_tokens)):
    net_network_trade_value[i] = net_network_trade_tokens[i] * market_value[i]

print(f"RECEIVED {net_network_trade_tokens[0]} TOKEN-0 = ${net_network_trade_value[0]}")
print(f"RECEIVED {net_network_trade_tokens[1]} TOKEN-1 = ${net_network_trade_value[1]}")
print(f"RECEIVED {net_network_trade_tokens[2]} TOKEN-2 = ${net_network_trade_value[2]}")
print(f"RECEIVED {net_network_trade_tokens[3]} TOKEN-3 = ${net_network_trade_value[3]}")

print(f"\nSUM OF RECEIVED TOKENS USD VALUE = ${sum(net_network_trade_value)}")
print(f"CONVEX OPTIMISATION SOLVER RESULT: ${prob.value}\n")



-------------------- ARBITRAGE TRADES + EXECUTION ORDER --------------------


TRADE POOL = BALANCER 0/1/2/3
	TENDERING 4.233521189139063 TOKEN 0
	TENDERING 0.13109434898192474 TOKEN 2
	RECEIVEING 2.135540503148681 TOKEN 1
	RECEIVEING 1.9283762038525643 TOKEN 3

TRADE POOL = UNIV2 0/1
	TENDERING 0.7363696474861502 TOKEN 1
	RECEIVEING 4.233521189567941 TOKEN 0

TRADE POOL = UNIV2 1/2
	TENDERING 0.2241902849074723 TOKEN 1
	RECEIVEING 0.9134224685462966 TOKEN 2

TRADE POOL = CONSTANT SUM 2/3
	TENDERING 3.867393209739356 TOKEN 3
	RECEIVEING 3.8635258157712378 TOKEN 2

TRADE POOL = UNIV2 2/3
	TENDERING 4.645853934648603 TOKEN 2
	RECEIVEING 5.189018026689082 TOKEN 3

-------------------- REQUIRED TOKENS TO KICK-START ARBITRAGE --------------------

TOKEN-0 = 4.233521189139063
TOKEN-1 = 0
TOKEN-2 = 0.13109434898192474
TOKEN-3 = 1.9390170058867917

USD VALUE REQUIRED = $12.42952149933282

-------------------- TOKENS & VALUE RECEIVED FROM ARBITRAGE --------------------

RECEIVED 4.288787014503